In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
from sklearn.model_selection import train_test_split

In [180]:
df = pd.read_csv('/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings/data_frc_1787_norm_text_with_embedding.csv')

In [181]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'subject', 'search_id',
       'dfate', 'physical-description', 'rerepublish', 'volume',
       'text_content', 'creator', 'sponsor', 'ocr', 'language', 'bwocr',
       'repub_state', 'languate', 'page-progression', 'scanfee', 'file_name',
       'contributor', 'year', 'invoice', 'link_to_catalog', 'foldoutcount',
       'scanner', 'uploader', 'republisher_date', 'collection',
       'physical_description', 'curation', 'item_number', 'scanningcenter',
       'openlibrary_edition', 'shiptracking', 'scandate', 'openlibrary_work',
       'sponsordate', 'date', 'repub_seconds', 'identifier', 'identifier-ark',
       'mediatype', 'notes', 'identifier-access', 'topicmodel', 'publisher',
       'republisher_operator', 'republisher_time', 'operator', 'ppi',
       'description', 'bookplateleaf', 'republisher', 'title', 'imagecount',
       'allsubjects', 'foldout_seconds', 'bookreader-defaults', 'lccn',
       'backup_location', 'call_number

In [182]:
df['topicmodel']

0      41, 42, 39
1      35, 39, 29
2      08, 35, 23
3      03, 34, 43
4      46, 39, 07
          ...    
126    49, 37, 35
127    08, 35, 15
128    08, 42, 03
129    39, 46, 40
130    05, 49, 16
Name: topicmodel, Length: 131, dtype: object

In [10]:
# Merging files to create dataset for classification tasks

directory = '/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings'
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

df_combined = pd.DataFrame()

for file in files:
    data = pd.read_csv(file, usecols=['date', 'call_number', 'openai_embedding', 'topicmodel']) 
    df_combined = pd.concat([df_combined, data], ignore_index=True)


In [11]:
df_combined.head(10)

,date,topicmodel,call_number,openai_embedding
0,1790,"20, 38, 36",Case folio FRC 9241,"[0.032563600689172745, -0.027827078476548195, ..."
1,1790,"23, 44, 02",Case FRC 289,"[0.031268347054719925, -0.031268347054719925, ..."
2,1790,"34, 29, 37",Case FRC 20150,"[0.03666583076119423, -0.06944313645362854, -0..."
3,1790,"01, 04, 03",Case FRC 16901,"[0.011695176362991333, -0.04674675688147545, -..."
4,1790,"29, 38, 35",Case folio FRC 27020,"[0.012459567748010159, -0.0034924547653645277,..."
5,1790,"34, 26, 28",Case FRC 18910,"[0.06023929640650749, -0.02725110948085785, -0..."
6,1790,"31, 39, 49",Case folio FRC 9962,"[0.02000940777361393, -0.005340734031051397, -..."
7,1790,"31, 39, 26",Case FRC 6086,"[0.003055283799767494, -0.019058657810091972, ..."
8,1790,"23, 42, 43",Case FRC 24426,"[0.014690271578729153, 0.0034852148965001106, ..."
9,1790,"23, 42, 11",Case FRC 1735,"[0.0032873915042728186, -0.0025112517178058624..."


In [12]:
len(df_combined)

5232

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer

df_combined['topicmodel'] = df_combined['topicmodel'].apply(lambda x: set([item.strip() for item in x.split(',')]))

# Applying MultiLabelBinarizer
mlb = MultiLabelBinarizer()
targets = mlb.fit_transform(df_combined['topicmodel'])

target_df = pd.DataFrame(targets, columns=mlb.classes_)

df_combined = pd.concat([df_combined, target_df], axis=1)

In [14]:
df_combined.head()

,date,topicmodel,call_number,openai_embedding,01,02,03,04,05,06,...,40,41,42,43,44,45,46,47,48,49
0,1790,"{36, 20, 38}",Case folio FRC 9241,"[0.032563600689172745, -0.027827078476548195, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1790,"{44, 23, 02}",Case FRC 289,"[0.031268347054719925, -0.031268347054719925, ...",0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1790,"{37, 29, 34}",Case FRC 20150,"[0.03666583076119423, -0.06944313645362854, -0...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1790,"{03, 04, 01}",Case FRC 16901,"[0.011695176362991333, -0.04674675688147545, -...",1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1790,"{35, 38, 29}",Case folio FRC 27020,"[0.012459567748010159, -0.0034924547653645277,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_combined.columns

Index(['date', 'topicmodel', 'call_number', 'openai_embedding', '01', '02',
       '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15',
       '16', '18', '19', '20', '22', '23', '25', '26', '27', '28', '29', '30',
       '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49'],
      dtype='object')

In [16]:
df_combined['00'] = 0

In [18]:
df_combined['14'] = 0
df_combined['17'] = 0
df_combined['21'] = 0
df_combined['24'] = 0

In [20]:
df_combined.to_csv('/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings/data_frc_1787_to_1791_with_embedding.csv', index=False)

In [21]:
# Creating train, test, val datasets

train_df, remaining_df = train_test_split(df_combined, test_size=0.3, random_state=43)
val_df, test_df = train_test_split(remaining_df, test_size=0.5, random_state=43)

In [22]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

3662
785
785


In [23]:
# Save data files

train_df.to_csv('/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings/frc_train_data.csv', index=False)
val_df.to_csv('/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings/frc_val_data.csv', index=False)
test_df.to_csv('/Users/ridhipurohit/Documents/French Revolution Data/Rep Learning/With Embeddings/frc_test_data.csv', index=False)

In [24]:
train_df.head()

,date,topicmodel,call_number,openai_embedding,01,02,03,04,05,06,...,45,46,47,48,49,00,14,17,21,24
81,1790,"{09, 30, 23}",Case FRC 2890,"[0.032517239451408386, -0.036904219537973404, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
815,1790,"{03, 30, 26}",Case FRC 19948,"[0.010577048175036907, -0.017527196556329727, ...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4381,1789,"{26, 02, 43}",Case FRC 1951,"[0.014622814953327179, -0.04139028117060661, -...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2429,1791,"{22, 49, 12}",Case FRC 14652,"[-0.03108709305524826, 0.007145257666707039, -...",0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5190,1789,"{46, 31, 23}",Case FRC 5679,"[-0.01994922384619713, 0.009577685967087746, -...",0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
